### DEPENDENCIES

In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import os
from time import perf_counter
import numpy as np
import inspect, itertools
import urllib
########################################################################

#---

### MAIN

In [24]:
driver = webdriver.Firefox()

# driver.get("http://www.python.org")
# assert "Python" in driver.title
# elem = driver.find_element(By.NAME, "q")
# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source
# driver.close()

login_url   = "https://www.linkedin.com/login/en"
password    = os.getenv('LINKEDIN_PASS')
user        = os.getenv('LINKEDIN_USER')

login_ret = login(driver    = driver,
      login_url = login_url,
      user      = user,
      password  = password
)

search_root_url = "https://www.linkedin.com/jobs/search/?"
keywords = "machine learning engineer"
location = "Brazil"
entry_level = True
remote_jobs = True
results = {}

for lang in search_terms["prog_langs"]:
      search_ret = search(    
      keywords    = lang,
      location    = "Worldwide",
      entry_level = False,
      remote_jobs = False,
      )
      results[lang] = search_ret
      #print(search_ret) # number of results

print(results)
driver.close()
#---

https://www.linkedin.com/jobs/search/?&geoId=92000000&keywords=python&location=Worldwide&refresh=true
search_query = 'PYTHON JOBS IN WORLDWIDE'
driver_title = 'PYTHON JOBS IN WORLDWIDE | LINKEDIN'
https://www.linkedin.com/jobs/search/?&geoId=92000000&keywords=java&location=Worldwide&refresh=true
search_query = 'JAVA JOBS IN WORLDWIDE'
driver_title = 'JAVA JOBS IN WORLDWIDE | LINKEDIN'
https://www.linkedin.com/jobs/search/?&geoId=92000000&keywords=javascript&location=Worldwide&refresh=true
search_query = 'JAVASCRIPT JOBS IN WORLDWIDE'
driver_title = 'JAVASCRIPT JOBS IN WORLDWIDE | LINKEDIN'
https://www.linkedin.com/jobs/search/?&geoId=92000000&keywords=ruby&location=Worldwide&refresh=true
search_query = 'RUBY JOBS IN WORLDWIDE'
driver_title = 'RUBY JOBS IN WORLDWIDE | LINKEDIN'
https://www.linkedin.com/jobs/search/?&geoId=92000000&keywords=go&location=Worldwide&refresh=true
search_query = 'GO JOBS IN WORLDWIDE'
driver_title = 'GO JOBS IN WORLDWIDE | LINKEDIN'
https://www.linkedin.com/jo

### LINKEDIN LOGIN

In [4]:
def login(driver, login_url, user, password, timeout_s=10):
    
    # login elements' xpath
    username_xpath = '//*[@id="username"]'
    password_xpath = '//*[@id="password"]'
    
    driver.get(login_url)
    assert "LinkedIn Login" in driver.title
    
    if "Welcome back" not in driver.page_source:
        username_elem = driver.find_element(By.XPATH, username_xpath)
        username_elem.send_keys(user)
    
    password_elem = driver.find_element(By.XPATH, password_xpath)

    password_elem.send_keys(password)
    password_elem.send_keys(Keys.RETURN)
    
    start = perf_counter()
    
    while "Feed" not in driver.title:
        sleep(1)
        
        timer_has_expired = (perf_counter() - start) > timeout_s
        
        if timer_has_expired:
            return False    
    
    return True
        
#---

### SEARCH

In [22]:
def search(    
    keywords,
    location    = "Worldwide",
    entry_level = False,
    remote_jobs = False,
):
    # print("Initiating search")
    
    worldwide       = location == "Worldwide"
    search_root_url = "https://www.linkedin.com/jobs/search/?"
    
    keywords_quoted = urllib.parse.quote(
        keywords, safe='/', encoding=None, errors=None)
    
    search_url = search_root_url + \
        ("f_E=2"            if entry_level  else "") + \
        ("&f_WT=2"          if remote_jobs  else "") + \
        ("&geoId=92000000"  if worldwide    else "") + \
        f"&keywords={keywords_quoted}" + \
        f"&location={location}" + \
        "&refresh=true"

    print(search_url)
    
    driver.get(search_url)
    
    search_query = f"{keywords} Jobs in {location}".upper()
    
    print(f"search_query = '{search_query}'" + \
        f"\ndriver_title = '{driver.title.upper()}'")
    
    assert search_query in driver.title.upper()

    no_of_results_elem = driver.find_element(By.TAG_NAME, "small")

    return no_of_results_elem.text.split()[0]

### FITLERS

In [7]:
all_filters = {
    
    "xp_level" : {
        "desc"          : "Experience Level",
        "prop"          : "f_E",
        "internship"    : 1,
        "entry_level"   : 2,
        "associate"     : 3,
        "mid_senior"    : 4,
        "director"      : 5,
        "executive"     : 6,
    },

    "dt_posted" : {
        "desc"      : "Date Posted",
        "prop"      : "f_TPR",
        "past_24H"  : "r86400",
        "past_week" : "r604800",
        "past_month": "r2592000",
    },

    "job_type" : {
        "desc"          : "Job Type",
        "prop"          : "f_JT",
        "fulltime"      : "F",
        "part_time"     : "P",
        "contract"      : "C",
        "temporary"     : "T",
        "volunteer"     : "V",
        "internship"    : "I",
        "other"         : "O",
    },

    "work_type" : {
        "desc"      : "On-site/Remote",
        "prop"      : "f_WT",
        "on_site"   : 1,
        "remote"    : 2,
        "hybrid"    : 3,
    },

    # @todo: map companies code
}

### SEARCH TERMS

In [6]:
search_terms = {
    "job_fields" : [
        "data",
        "data science",
        "machine learning",
        "deep learning",
        "cloud",
        "analytics"
    ],
    "prog_langs" : [
        "python",
        "java",
        "javascript",
        "ruby",
        "go",
        "c",
        "c++",
        "c#",
    ],
    "job_titles" : [
        "engineer",
        "specialist"
        "developer",
        "analyst",
    ]
}